# Modelo de Aprendizaje Automatico

Tras concluir el Análisis Exploratorio de Datos (EDA) para comprender la naturaleza de nuestros datos, se procede con la fase siguiente: la creación de un modelo de recomendación basado en la similitud del coseno. 

Para el primer enfoque del modelo, se establece una relación ítem-ítem. En este escenario, se evalúa un ítem con respecto a su similitud con otros ítems para ofrecer recomendaciones similares. En este caso, el input corresponde a un juego y el output es una lista de juegos recomendados, utilizando el concepto de similitud del coseno.

Por otra parte, se considera una segunda propuesta para el sistema de recomendación, basada en el filtro user-item. En esta estrategia, se analiza a un usuario para identificar usuarios con gustos similares y se recomiendan ítems que hayan sido apreciados por estos usuarios afines.


In [138]:
import pandas as pd # Importo las libreias que utilizare en este proceso
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity


- Previo al inicio de mi modelo, realicé una conversión a un nuevo DataFrame, incluyendo únicamente las columnas necesarias para llevar a cabo este proceso.

In [2]:
item_model= pd.read_csv('modeloo_item.csv', low_memory=False) # Leó el Dataframe llamado 'modeloo_item' 


In [3]:
item_model.head() # Visualizo nada mas para revisar su forma.

,user_id,item_id,genres,app_name
0,wolfclawe,22330,NaN,NaN
1,SirTrollbuscus,323320,NaN,NaN
2,Gottheart,345390,NaN,NaN
3,76561198060988138,253710,NaN,NaN
4,76561198004058147,320,NaN,NaN


In [4]:
item_model.shape # Acá solo pra corroborar cuantas columnas tengo y hacerme una idea de como trabajarlos.

(80000, 4)

- Creo un dataframe que contenga las columnas de generos para poder combinarlas

In [5]:
# Creación de dummies
item_model = pd.get_dummies(item_model, columns=['genres'], prefix="")

# Agrupación por 'user_id' e 'item_id', sumando las columnas
item_model = item_model.groupby(['item_id','app_name']).sum().reset_index()

item_model.head()



,item_id,app_name,user_id,_Action,_Action Adventure,_Action Adventure Casual Free to Play Indie RPG,_Action Adventure Casual Indie,_Action Adventure Casual Indie RPG Strategy,_Action Adventure Casual Indie Strategy Early Access,_Action Adventure Free to Play Massively Multiplayer RPG,...,_RPG Strategy,_Racing Simulation,_Racing Simulation Sports,_Racing Sports,_Simulation,_Simulation Sports,_Simulation Strategy,_Simulation Strategy Early Access,_Sports,_Strategy
0,10,Pool Nation FX Lite,jonasdbomb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80,The Aquatic Adventure of the Last Human Delux...,76561198089393905,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100,Manyland,jonasdbomb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,240,The Ritual on Weylyn Island,jonasdbomb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,280,BOROS,sad-commie,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
item_model.shape

(334, 117)

- Calculo la similitud del coseno

In [7]:
coseno_similit = cosine_similarity(item_model.iloc[:,3:])

In [8]:
coseno_similit.shape

(334, 334)

In [9]:
print(coseno_similit)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 1.]]


- Hago la funcion que me devolvera los juegos recomendados. 

In [108]:
def recomendacion_juegos(item_id):
    item_id = int(item_id)
    
    # Aqui lo que hare es como igualarlo comforme al id del juego.
    seleccion_juego = item_model[item_model['item_id'] == item_id]
    
    # Devolver error en caso de vacío
    if seleccion_juego.empty:
        return "El juego con el item especificado no existe en la base de datos."
    
    # Calcula la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = coseno_similit[item_model[item_model['item_id'] == item_id].index[0]]
    
    # Calcula los índices de los juegos más similares (excluyendo el juego de entrada)
    juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = item_model.iloc[juegos_similares]['app_name']
    
    return list(juegos_recomendados)


In [109]:
recomendados = recomendacion_juegos(280)
recomendados


['Super Space Pug', 'Miner Mayhem', 'Inertia', ' Ride', 'Titan Attacks']

In [110]:
item_model.head()

,item_id,app_name,user_id,_Action,_Action Adventure,_Action Adventure Casual Free to Play Indie RPG,_Action Adventure Casual Indie,_Action Adventure Casual Indie RPG Strategy,_Action Adventure Casual Indie Strategy Early Access,_Action Adventure Free to Play Massively Multiplayer RPG,...,_RPG Strategy,_Racing Simulation,_Racing Simulation Sports,_Racing Sports,_Simulation,_Simulation Sports,_Simulation Strategy,_Simulation Strategy Early Access,_Sports,_Strategy
0,10,Pool Nation FX Lite,jonasdbomb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80,The Aquatic Adventure of the Last Human Delux...,76561198089393905,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100,Manyland,jonasdbomb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,240,The Ritual on Weylyn Island,jonasdbomb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,280,BOROS,sad-commie,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- Voy a dividir el dataframe para poder subir en render. 

In [59]:
#Cuento filas
filas= len(item_model)

#Calculo la mitad
mitad_filas= filas // 10
 
#Selecciono la mitad superior
render_model= item_model.iloc[:mitad_filas]

In [128]:
render_model.shape

(33, 117)

In [129]:
coseno_render = cosine_similarity(item_model.iloc[:,3:])

- Lo convierto a un nuevo archivo csv para render.

In [130]:
render_model.to_csv('modeloo_render.csv',index=False)

In [131]:
render_model

,item_id,app_name,user_id,_Action,_Action Adventure,_Action Adventure Casual Free to Play Indie RPG,_Action Adventure Casual Indie,_Action Adventure Casual Indie RPG Strategy,_Action Adventure Casual Indie Strategy Early Access,_Action Adventure Free to Play Massively Multiplayer RPG,...,_RPG Strategy,_Racing Simulation,_Racing Simulation Sports,_Racing Sports,_Simulation,_Simulation Sports,_Simulation Strategy,_Simulation Strategy Early Access,_Sports,_Strategy
0,10,Pool Nation FX Lite,jonasdbomb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80,The Aquatic Adventure of the Last Human Delux...,76561198089393905,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100,Manyland,jonasdbomb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,240,The Ritual on Weylyn Island,jonasdbomb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,280,BOROS,sad-commie,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,300,Beach Rules,76561197970982479,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,360,MX vs ATV Supercross Encore Suzuki RMZ MX,76561198089393905,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7,360,Street Fighter X Tekken Street Fighter Boost G...,POOTISSPENSOR,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,360,Trump Simulator,ABSOLUTLY_NOTHING,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,380,HITMAN Blood Money Requiem Pack,PPanther,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- Primera Función item-item, introduzco el id del juego y me devuelve juegos recomendados.

In [139]:
def encontrar_juegos(item_id):
    # Encuentra el índice del juego ingresado por item_id
    juego_indice = render_model.index[render_model['item_id'] == item_id].tolist()[0]
    
    # Extraigo las características del juego 
    juego_caracteristicas = render_model.iloc[juego_indice, 3:].values.reshape(1, -1)
    
    # Y acá se calcula la similitud coseno entre el juego ingresado y todos los demás juegos
    render = cosine_similarity(render_model.iloc[:, 3:], juego_caracteristicas)
    
    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    lineas_juegos_similares = render.argsort(axis=0)[::-1][1:6]
    lineas_juegos_similares = lineas_juegos_similares.flatten()[1:]
    
    # Obtiene los juegos más similares en función de los índices
    juegos_similares = render_model.iloc[lineas_juegos_similares]['app_name']
    
    return juegos_similares

In [140]:
juegos_similares = encontrar_juegos(8870)
print(juegos_similares)

1    The Aquatic Adventure of the Last Human  Delux...
2                                             Manyland
3                          The Ritual on Weylyn Island
4                                                BOROS
Name: app_name, dtype: object


- Segunda Función de usuario-item, Ingreso el id  del usuario y le devuelve juegos recomendados.

In [135]:
def recomendacion_usuario(user_id, render_model):
    # Encuentra con el user_id los juegos recomendados
    if user_id in render_model['user_id'].values:
        juegos = render_model.index[render_model['user_id'] == user_id].tolist()[0]
        
        juego_caracteristicas = render_model.iloc[juegos, 3:].values.reshape(1, -1)
        
        render_similitud = cosine_similarity(render_model.iloc[:, 3:], juego_caracteristicas)
        juegos_similaresrecomend = render_similitud.argsort(axis=0)[::-1][1:6]
        juegos_similaresrecomend = juegos_similaresrecomend.flatten()[1:]
        juegos_similares = render_model.iloc[juegos_similaresrecomend]['app_name']
        
        return juegos_similares  
    else:
        return "El juego con el user_id especificado no existe en la base de datos."
    
user_id = 'sad-commie'
juegos_recomendados = recomendacion_usuario(user_id, render_model)

print(juegos_recomendados) 


26                 Ride
32    Mysterious Castle
8      Trump Simulator 
12        Lethal League
Name: app_name, dtype: object
